In [ ]:
import proveit.special_statement_magic # for %begin_axioms and %end_axioms
from proveit.expression import Operation
from proveit.basiclogic import Forall, In, Equals
from proveit.numbers import Add, subtract, Multiply, Exponentiate, frac, Abs, Floor
from proveit.numbers import greaterThanEquals, LessThanEquals, Mod, ModAbs, Round, IntervalCO, DiscreteContiguousSet
from proveit.numbers.common import e, i, zero, one, two, pi, two_pi, Integer, NaturalPos, Real
from proveit.numbers import Natural
from proveit.trigonometry import AngDiff
from proveit.linalg import MatrixProd, ScalarProd, SU, TensorProd, TensorExp
from proveit.statistics import Prob
from proveit.physics.quantum import Bra, Ket, RegisterBra, RegisterKet, Meas, Circuit, Gate, Input, Output, MultiWire
from proveit.physics.quantum.common import Hgate, RegisterSU, QubitRegisterSpace, ket0, \
    CTRL_DN, WIRE_LINK, WIRE_DN, PASS
from proveit.physics.quantum.QFT import InverseFourierTransform
# Note that U_, u_, and t_ are Literals that will eventually be converted to
# Variable counterparts through meta-reasoning.
from proveit.physics.quantum.QPE.common import U_, n_, u_, phase_, t_, Psi_, m_, phase_m_, b_, delta_, two_pow_t,\
    psi_1, psi_k, psi_next, psi_t, alpha_l, U_pow_two_pow_k
from proveit.physics.quantum.QPE import QPE, PhaseEst
from proveit.physics.quantum.QPE.phase_est_ops import Psuccess, Pfail, ModAdd
from proveit.common import U, u, t, eps, a, b, k, l
from IPython.display import display

In [ ]:
%begin_axioms

Let $U$ be a unitary operator that acts on $n$ qubits, $u$ an eigenstate of U with eigenvalue $e^{2 \pi i \varphi}$:

In [ ]:
n_in_nat_pos = In(n_, NaturalPos) # transferred
unitary_u = In(U_, RegisterSU(n_))  # transferred
u_ket_register = In(Ket(u_), QubitRegisterSpace(n_))  # transferred
phase_in_interval = In(phase_, IntervalCO(zero, one)) # transferred
eigen_uu = Equals(MatrixProd(U_, Ket(u_)), ScalarProd(Exponentiate(e, Multiply(two, pi, i, phase_)), Ket(u_))) # transferred
display(n_in_nat_pos)
display(unitary_u)
display(u_ket_register)
display(phase_in_interval)
display(eigen_uu)  

Define the Quantum Phase Estimation algorithm, for any $U$, $u$, and $t$ (number of bits of precision), using quantum circuit representations, broken into three stages with recursion in the middle.

In [ ]:
t_in_natural_pos = In(t_, NaturalPos) # transferred
psi_kets_in_q_reg_space = Forall(k, In(Ket(psi_k), QubitRegisterSpace(k)), domain=NaturalPos) # transferred
PsiKetInQRegSpace = In(Ket(Psi_), QubitRegisterSpace(t_)) # transferred
display(t_in_natural_pos)
display(psi_kets_in_q_reg_space)
display(PsiKetInQRegSpace)

In [ ]:
first_q_p_e = Circuit([[Input(ket0), Hgate, CTRL_DN, Output(Ket(psi_1))],
                [Input(Ket(u_)), MultiWire(n_), Gate(U_), Output(Ket(u_))]])
first_q_p_e

In [ ]:
mid_q_p_e = Forall(k, 
               Circuit([[Input(Ket(psi_k)), MultiWire(k), PASS, WIRE_DN, PASS, PASS],
                [Input(ket0), Hgate, CTRL_DN, WIRE_LINK, MultiWire(Add(k, one)), Output(Ket(psi_next))],
                [Input(Ket(u_)), MultiWire(n_), Gate(U_pow_two_pow_k), Output(Ket(u_)), PASS, PASS]]),
       domain=NaturalPos)
mid_q_p_e

In [ ]:
final_q_p_e = Circuit([[Input(Ket(psi_t)), MultiWire(t_), Gate(InverseFourierTransform(t_)), Output(Ket(Psi_))]])
final_q_p_e

Let $\lvert \Psi \rangle$ be the outcome of the $t$-qubit register of ${\rm QPE}(U, t)$ acting on $\lvert u \rangle$, $m$ be a random variable representing the measurement of Psi with the register interpretted as an integer (via binary representation), and $\varphi_m = 2 \pi m/2^t$ be the random variable phase outcome of ${\rm QPE}(U, t)$:

In [ ]:
m_def = Equals(m_, Meas(Ket(Psi_))) # transferred
phase_m_def = Equals(phase_m_, frac(m_, two_pow_t)) # transferred
display(m_def)
display(phase_m_def)

Let $b$ be the value for $m$ that gives the closest $\varphi_m$ to $\varphi$ without exceeding it, and let $\delta$ be this difference:

In [ ]:
best_def = Equals(b_, Floor(Multiply(phase_, two_pow_t))) # transferred
delta_def = Equals(delta_, subtract(phase_, frac(b_, two_pow_t))) # transferred
display(best_def)
display(delta_def)

The probability of success is defined as $\theta_m$ being within some epsilon of $\theta$: 

In [ ]:
# transferred
success_def = Forall(eps, Equals(Psuccess(eps), 
                                Prob(LessThanEquals(ModAbs(subtract(m_, b_), two_pow_t), eps), m_)), 
                    domain=NaturalPos)
# transferred
fail_def = Forall(eps, Equals(Pfail(eps), subtract(one, Psuccess(eps))), domain=NaturalPos)
display(success_def)
display(fail_def)             

Let $\alpha_l$ be the amplitude of $\lvert \Psi \rangle$ for the $b \oplus l$ state, where $\oplus$ is defined as addition modulo $2^t$:

In [ ]:
# transferred
mod_add_def = Forall((a, b), Equals(ModAdd(a, b), Mod(Add(a, b), two_pow_t)), domain=Integer)
# transferred
alpha_def = Forall(l, Equals(alpha_l, MatrixProd(RegisterBra(ModAdd(b_, l), t_), Ket(Psi_))), domain=Integer)
display(mod_add_def)
display(alpha_def)

In [ ]:
%end_axioms